# Тестирование функций и методов
---

### Модуль Ingestion (Docling)

#### Загрузка и конвертация документа

In [1]:
import src.document_loader as document_loader
import config.config as cfg

def test_load_document():
    file_path = str(cfg.RAW_DATA_DIR / "sample.pdf")
    result = document_loader.load_document(file_path)
    print(f"Result_doc: {result[0:100]}...")  # Print first 100 characters for brevity

if __name__ == "__main__":
    test_load_document()

d:\Projects\LocalRAG\.ven\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[INFO] 2026-01-20 16:30:23,816 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2026-01-20 16:30:23,824 [RapidOCR] device_config.py:50: Using CPU device
[INFO] 2026-01-20 16:30:23,836 [RapidOCR] download_file.py:60: File exists and is valid: D:\Projects\LocalRAG\.ven\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_det_infer.pth
[INFO] 2026-01-20 16:30:23,837 [RapidOCR] main.py:50: Using D:\Projects\LocalRAG\.ven\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_det_infer.pth
[INFO] 2026-01-20 16:30:24,133 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2026-01-20 16:30:24,134 [RapidOCR] device_config.py:50: Using CPU device
[INFO] 2026-01-20 16:30:24,137 [RapidOCR] download_file.py:60: File exists and is valid: D:\Projects\LocalRAG\.v

Saved intermediate file: data\processed\sample.md
Result_doc: 1. Научные открытия и технологические прорывы

В 2023 году учёные из ЦЕРНа подтвердили существование...


#### Чанкирование по кол-ву симовлов с перекрытием

In [2]:
from src.chunker import chunk_text
from config.config import PROCESSED_DATA_DIR

file_path = PROCESSED_DATA_DIR / "sample.md"
markdown_content = file_path.read_text(encoding="utf-8")

CHUNK_SIZE = 500 
OVERLAP = 100
file_name = file_path.name # Сохраняем имя для Qdrant

chunks = chunk_text(markdown_content, chunk_size=CHUNK_SIZE, overlap=OVERLAP)

print(f"Файл '{file_name}' успешно разбит.")
print(f"Количество чанков: {len(chunks)}")
print(f"Пример первого чанка:\n{repr(chunks[0][:100])}...")

Файл 'sample.md' успешно разбит.
Количество чанков: 12
Пример первого чанка:
'1. Научные открытия и технологические прорывы\n\nВ 2023 году учёные из ЦЕРНа подтвердили существование'...


#### Сохранение векторизованных данных в Qdrant

In [3]:
from src.embedder import vectorize_and_upload

q_client, embed_model = vectorize_and_upload(chunks, file_name)

Генерация векторов для 12 чанков...
Данные успешно загружены в коллекцию 'docs_collection'


### Модули Retrieval и Reranker

#### Retrieval

In [4]:
from src.retrieval import retrieve

user_query = "Скольки градусам может достичь потепление к 2050 году?"
candidates = retrieve(user_query, q_client, embed_model, top_k=3)

Найдено 3 кандидатов через векторный поиск.


In [5]:
print("Топ-3 кандидата по запросу:")
for i, candidate in enumerate(candidates, start=1):
    print(f"{i}. {candidate}\n")

Топ-3 кандидата по запросу:
1. {'text': 'аду МГЭИК (Межправительственной группы экспертов по изменению климата) за 2022 год, глобальная средняя температура уже на 1,15°C выше доиндустриального уровня. Если выбросы парниковых газов не будут радикально сокращены, к 2050 году потепление может достичь 2,7°C, что приведёт к катастрофическим последствиям: повышению уровня моря, экстремальным погодным явлениям и массовому вымиранию видов.\n\nОднако существуют и контраргументы: некоторые исследователи, такие как климатолог Бьорн Ломборг, утвержд', 'score': 0.7725014185102526, 'metadata': {'source': 'sample.md', 'chunk_id': 5}}

2. {'text': 'ко существуют и контраргументы: некоторые исследователи, такие как климатолог Бьорн Ломборг, утверждают, что адаптация к изменениям климата может быть более эффективной стратегией, чем полный отказ от ископаемого топлива. Тем не менее, большинство научного сообщества поддерживает Парижское соглашение 2015 года, целью которого является удержание роста темпер

#### Reranker

In [6]:
from src.reranker import rerank

final_context = rerank(user_query, candidates, top_n=3)

# Вывод результата
for i, res in enumerate(final_context):
    print(f"\n--- Релевантный чанк №{i+1} (Score: {res['rerank_score']:.4f}) ---")
    print(res["text"][:200] + "...")

Переранжирование завершено. Выбрано топ-3 наиболее релевантных.

--- Релевантный чанк №1 (Score: 8.6397) ---
аду МГЭИК (Межправительственной группы экспертов по изменению климата) за 2022 год, глобальная средняя температура уже на 1,15°C выше доиндустриального уровня. Если выбросы парниковых газов не будут р...

--- Релевантный чанк №2 (Score: 8.4987) ---
 содержащих стереотипы, она может их воспроизводить.

В то же время ИИ активно применяется в медицине: алгоритмы DeepMind (подразделение Google) помогают диагностировать диабетическую ретинопатию и ра...

--- Релевантный чанк №3 (Score: 7.9926) ---
ко существуют и контраргументы: некоторые исследователи, такие как климатолог Бьорн Ломборг, утверждают, что адаптация к изменениям климата может быть более эффективной стратегией, чем полный отказ от...


In [7]:
final_context[0]["text"]

'аду МГЭИК (Межправительственной группы экспертов по изменению климата) за 2022 год, глобальная средняя температура уже на 1,15°C выше доиндустриального уровня. Если выбросы парниковых газов не будут радикально сокращены, к 2050 году потепление может достичь 2,7°C, что приведёт к катастрофическим последствиям: повышению уровня моря, экстремальным погодным явлениям и массовому вымиранию видов.\n\nОднако существуют и контраргументы: некоторые исследователи, такие как климатолог Бьорн Ломборг, утвержд'

### Модуль Generator

#### Генератор с использованием локальной модели

In [8]:
from src.generator import Generator

generator = Generator()

user_query = "Скольки градусам может достичь потепление к 2050 году?"
response = generator.generate(user_query, final_context[0]["text"])

print("\n--- Сгенерированный ответ ---")
print(response)


--- Сгенерированный ответ ---
Согласно предоставленному контексту, потепление может достичь уровня 2,7°C к 2050 году.


Получение итогового списка контекста модели

In [9]:
final_context_text = []
for i in range(len(final_context)):
    final_context_text.append(final_context[i]["text"])

In [10]:
print(final_context_text)

['аду МГЭИК (Межправительственной группы экспертов по изменению климата) за 2022 год, глобальная средняя температура уже на 1,15°C выше доиндустриального уровня. Если выбросы парниковых газов не будут радикально сокращены, к 2050 году потепление может достичь 2,7°C, что приведёт к катастрофическим последствиям: повышению уровня моря, экстремальным погодным явлениям и массовому вымиранию видов.\n\nОднако существуют и контраргументы: некоторые исследователи, такие как климатолог Бьорн Ломборг, утвержд', ' содержащих стереотипы, она может их воспроизводить.\n\nВ то же время ИИ активно применяется в медицине: алгоритмы DeepMind (подразделение Google) помогают диагностировать диабетическую ретинопатию и рак молочной железы с точностью, сопоставимой с экспертами-людьми.\n\n## 5. География и демография\n\nНаселение Земли достигло 8 миллиардов человек в ноябре 2022 года. По прогнозам ООН, пик будет достигнут около 2086 года - на уровне 10,4 миллиарда. При этом старение населения становится сер

---
### Evaluation (Ragas)

#### Rertieaval evaluation

#### Genearator evaluation

##### - Faithfulness

Faithfulness metrics — насколько ответ соответствует фактам из ретривнутых документов

In [19]:
from src.evaluation.faithfulness import getting_faithfulness_metric
import asyncio

faithfulness_metric = await getting_faithfulness_metric("gpt-oss:20b", user_query, response, final_context_text)
print(f"Faithfulness: {faithfulness_metric}")

Faithfulness: 1.0


##### - Response Relevancy

In [ ]:
from src.evaluation.response_relevancy import getting_resp_rel_metric
from sentence_transformers import SentenceTransformer
import asyncio

# Пример вызова
result = await getting_resp_rel_metric(
    judje_model="gpt-oss:20b",
    embeddings_model_name="sentence-transformers/all-MiniLM-L6-v2",
    user_input=user_query,
    response=response,
    retrieved_context=final_context_text
)

print(result)

LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.


AttributeError: 'SentenceTransformer' object has no attribute 'embed_query'